# Week-18 Homework Assignment

#### 1.	What is a neural network? What are the general steps required to build a neural network? 

#### Neural network
A large connected network which is passing the inputs(values) through hidden layers and there is some computation happening
to get the desired output. speeding up the process with high efficiency is one of the advantage in artificial neural networks.

#### 2.	Generally, how do you check the performance of a neural network? Why? 

#### 3.	Create a neural network using keras to predict the outcome of either of these datasets: 

Cardiac Arrhythmia: https://archive.ics.uci.edu/ml/datasets/Arrhythmia 

Abalone age: https://archive.ics.uci.edu/ml/datasets/Abalone

The common name for Abalone is sea snail. we can determine the age of the abalone by cutting the shell through the cone,
stained and the rings are counted using the microscope. All these is a time consuming process that can be simplified 
using neural networks. Here, we can predict their age using physical measurements like length, height, weight and
other features.

Lets import our Abalone dataset and some dependent libraries

In [378]:
import pandas as pd 
import numpy as np

In [379]:
# import the dataset (Abalone dataset)
target_url = (r"A:\launch_code_STL\Final_Homework\week-17\abalone.data")

In [380]:
abalone_df = pd.read_csv(target_url,header=None, prefix="V")
abalone_df.columns = ['Sex', 'Length', 'Diameter', 'Height',
                   'Whole weight', 'Shucked weight',
                   'Viscera weight', 'Shell weight', 'Rings']

print(abalone_df.shape)
print(abalone_df.head())

(4177, 9)
  Sex  Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
4   I   0.330     0.255   0.080        0.2050          0.0895          0.0395   

   Shell weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
4         0.055      7  


The dataset contains 4177 samples and each sample has 9 features('Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 
'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings') The Rings feature is the target variable and as given 
in the documentation, adding 1.5 to the ring attribute gives the age of the abalone.

###  Preprocessing

#### Encoding the input columns

Out of the 8 input columns, 'Sex' column is a categorical attribute, we need to perform one hot encoding, we use 
get_dummies() function.

In [381]:
# One hot encoding the sex attribute.
df_dummies = pd.get_dummies(abalone_df['Sex'], prefix = "Sex_") # drop_first parameter is set to True, this means
                                                                           # that if there are n categories in the column,
                                                                           # n-1 columns are returned instead of n.

# Inserting dummy columns
for column in df_dummies.columns:
    abalone_df[column] = df_dummies[column]  # one-hot encoded columns are appended to the data frame
    
# Dropping the original column
abalone_df = abalone_df.drop(columns = ['Sex'])     # original 'Sex' attribute column is dropped

In [382]:
print(abalone_df)

      Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0      0.455     0.365   0.095        0.5140          0.2245          0.1010   
1      0.350     0.265   0.090        0.2255          0.0995          0.0485   
2      0.530     0.420   0.135        0.6770          0.2565          0.1415   
3      0.440     0.365   0.125        0.5160          0.2155          0.1140   
4      0.330     0.255   0.080        0.2050          0.0895          0.0395   
...      ...       ...     ...           ...             ...             ...   
4172   0.565     0.450   0.165        0.8870          0.3700          0.2390   
4173   0.590     0.440   0.135        0.9660          0.4390          0.2145   
4174   0.600     0.475   0.205        1.1760          0.5255          0.2875   
4175   0.625     0.485   0.150        1.0945          0.5310          0.2610   
4176   0.710     0.555   0.195        1.9485          0.9455          0.3765   

      Shell weight  Rings  Sex__F  Sex_

#### Encoding the output columns

The attribute 'Rings' is with continuous values ranging from 1 to 29. we can split these into 3 categories — 
less than 10(young), between 10 and 20 (middle age), between 20 and 30(old).


In [383]:
def rings_label(x):
    if x<=10:
        return 'young'
    if x<=20:
        return 'middle age'
    if x<=30:
        return 'old'
    
abalone_df['Rings'] = abalone_df['Rings'].apply(rings_label)

In [384]:
print(abalone_df)

      Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0      0.455     0.365   0.095        0.5140          0.2245          0.1010   
1      0.350     0.265   0.090        0.2255          0.0995          0.0485   
2      0.530     0.420   0.135        0.6770          0.2565          0.1415   
3      0.440     0.365   0.125        0.5160          0.2155          0.1140   
4      0.330     0.255   0.080        0.2050          0.0895          0.0395   
...      ...       ...     ...           ...             ...             ...   
4172   0.565     0.450   0.165        0.8870          0.3700          0.2390   
4173   0.590     0.440   0.135        0.9660          0.4390          0.2145   
4174   0.600     0.475   0.205        1.1760          0.5255          0.2875   
4175   0.625     0.485   0.150        1.0945          0.5310          0.2610   
4176   0.710     0.555   0.195        1.9485          0.9455          0.3765   

      Shell weight       Rings  Sex__F 

In [385]:
# One hot encoding the Rings attribute.
df_dummies = pd.get_dummies(abalone_df['Rings'])

# Inserting dummy columns
for column in df_dummies.columns:
    abalone_df[column] = df_dummies[column]
    
# Dropping the original column
df = abalone_df.drop(columns = ['Rings'])

In [386]:
# defining the input and output columns to separate the dataset in the later cells.

input_columns = df.columns.tolist()
input_columns.remove('young')
input_columns.remove('middle age')
input_columns.remove('old')

output_columns = ['young', 'middle age', 'old']

In [387]:
print(df)

      Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0      0.455     0.365   0.095        0.5140          0.2245          0.1010   
1      0.350     0.265   0.090        0.2255          0.0995          0.0485   
2      0.530     0.420   0.135        0.6770          0.2565          0.1415   
3      0.440     0.365   0.125        0.5160          0.2155          0.1140   
4      0.330     0.255   0.080        0.2050          0.0895          0.0395   
...      ...       ...     ...           ...             ...             ...   
4172   0.565     0.450   0.165        0.8870          0.3700          0.2390   
4173   0.590     0.440   0.135        0.9660          0.4390          0.2145   
4174   0.600     0.475   0.205        1.1760          0.5255          0.2875   
4175   0.625     0.485   0.150        1.0945          0.5310          0.2610   
4176   0.710     0.555   0.195        1.9485          0.9455          0.3765   

      Shell weight  Sex__F  Sex__I  Sex

In [326]:
# Splitting into train, val and test set -- 80-10-10 split
from sklearn.model_selection import train_test_split

# First, an 80-20 split
train_df, val_test_df = train_test_split(df, test_size = 0.2)

# Then split the 20% into half
val_df, test_df = train_test_split(val_test_df, test_size = 0.5)

# Splitting into X (input) and y (output)

X_train, y_train = np.array(train_df[input_columns]), np.array(train_df[output_columns])

X_val, y_val = np.array(val_df[input_columns]), np.array(val_df[output_columns])

X_test, y_test = np.array(test_df[input_columns]), np.array(test_df[output_columns])

In [327]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

Xtrain = sc.fit_transform(X_train)
Xval = sc.transform(X_val)
Xtest = sc.transform(X_test)

In [328]:
# Import necessary modules
import keras
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras import layers
import keras.callbacks
from keras.callbacks import EarlyStopping


model = Sequential([
    layers.Dense(32, activation = 'relu', input_shape = X_train[0].shape),
    layers.Dense(8, activation = 'relu'),
    layers.Dense(3, activation = 'softmax')])

early_stopping_monitor = EarlyStopping(patience=2)

model.compile(optimizer = 'Adam', loss = 'mean_absolute_error', metrics = ['mean_absolute_error'])

# model.compile(optimizer = 'Adam', loss = 'CategoricalCrossentropy', metrics = ['accuracy'])

history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 256, callbacks = [early_stopping_monitor])

Epoch 1/256
105/105 [==============================] - 0s 2ms/step - loss: 0.3443 - mean_absolute_error: 0.3443 - val_loss: 0.2598 - val_mean_absolute_error: 0.2598
Epoch 2/256
105/105 [==============================] - 0s 1ms/step - loss: 0.2370 - mean_absolute_error: 0.2370 - val_loss: 0.2376 - val_mean_absolute_error: 0.2376
Epoch 3/256
105/105 [==============================] - 0s 1ms/step - loss: 0.2298 - mean_absolute_error: 0.2298 - val_loss: 0.2357 - val_mean_absolute_error: 0.2357
Epoch 4/256
105/105 [==============================] - 0s 1ms/step - loss: 0.2287 - mean_absolute_error: 0.2287 - val_loss: 0.2351 - val_mean_absolute_error: 0.2351
Epoch 5/256
105/105 [==============================] - 0s 1ms/step - loss: 0.2284 - mean_absolute_error: 0.2284 - val_loss: 0.2349 - val_mean_absolute_error: 0.2349
Epoch 6/256
105/105 [==============================] - 0s 1ms/step - loss: 0.2282 - mean_absolute_error: 0.2282 - val_loss: 0.2347 - val_mean_absolute_error: 0.2347
Epoch 7/25

In [308]:
model.evaluate(X_test, y_test)

14/14 [==============================] - 0s 1ms/step - loss: 0.2472 - mean_absolute_error: 0.2472


[0.24720904231071472, 0.24720904231071472]

In [309]:
# Model summary for number of parameters used in the algorithm
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 32)                320       
                                                                 
 dense_19 (Dense)            (None, 8)                 264       
                                                                 
 dense_20 (Dense)            (None, 3)                 27        
                                                                 
Total params: 611
Trainable params: 611
Non-trainable params: 0
_________________________________________________________________


#### 4.	Write another algorithm to predict the same result as the previous question using either KNN or logistic regression.

In [345]:
abalone_df = pd.read_csv(target_url,header=None, prefix="V")
abalone_df.columns = ['Sex', 'Length', 'Diameter', 'Height',
                   'Whole weight', 'Shucked weight',
                   'Viscera weight', 'Shell weight', 'Rings']
# abalone_df = pd.DataFrame(abalone_df, columns=abalone_df.columns)
# y = abalone_df.Rings

print(abalone_df.shape)
print(abalone_df.head())

(4177, 9)
  Sex  Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
4   I   0.330     0.255   0.080        0.2050          0.0895          0.0395   

   Shell weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
4         0.055      7  


In [346]:
sex_dictionary = {'M': 1, 'F' : 2,'I':3}
abalone_df['Sex'] = abalone_df['Sex'].apply(lambda x: sex_dictionary[x])

In [347]:
abalone_df.dtypes

Sex                 int64
Length            float64
Diameter          float64
Height            float64
Whole weight      float64
Shucked weight    float64
Viscera weight    float64
Shell weight      float64
Rings               int64
dtype: object

In [348]:
abalone_df.select_dtypes(include=['object']).isnull().sum()
abalone_df.select_dtypes(include=[np.number]).isnull().sum()

Sex               0
Length            0
Diameter          0
Height            0
Whole weight      0
Shucked weight    0
Viscera weight    0
Shell weight      0
Rings             0
dtype: int64

In [349]:
# Adding age column in the dataframe for classification. Age is classified based on the number of rings.
#Rings 1-8 -->Age 1, denoting young
# Rings 9-10 -->Age 2, denoting middle
# Rings 11-29 -->Age 3, denoting old

In [350]:
def label_race (row):
    if row['Rings']>=1 and row['Rings']<=8:
        return 1
    elif row['Rings'] >=9 and row['Rings']<=10:
        return 2
    elif row['Rings']>=11 and row['Rings']<=29:
        return 3
    return 0

In [351]:
abalone_df['Age'] = abalone_df.apply(lambda _: '', axis=1)
abalone_df['Age']=abalone_df.apply (lambda row: label_race (row),axis=1)
# abalone_df.Sex = abalone_df.Sex.astype(float)
abalone_df

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Age
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15,3
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7,1
2,2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9,2
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10,2
4,3,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7,1
...,...,...,...,...,...,...,...,...,...,...
4172,2,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11,3
4173,1,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10,2
4174,1,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9,2
4175,2,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10,2


In [352]:
abalone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   int64  
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole weight    4177 non-null   float64
 5   Shucked weight  4177 non-null   float64
 6   Viscera weight  4177 non-null   float64
 7   Shell weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
 9   Age             4177 non-null   int64  
dtypes: float64(7), int64(3)
memory usage: 326.5 KB


In [127]:
# Response variable is age

In [353]:
y = abalone_df[['Age']]
X = abalone_df.drop(['Age','Rings'], axis=1)

In [354]:
# Dividing the dataset into 75% training and 25% test data using train_test_split

# from sklearn import cross_validation
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_validate as cv

# X_train, y_train, X_test, y_test =train_test_split(X, y)
train_X, test_X, train_y, test_y = train_test_split(X, y) # splits 75%/25% by default

In [355]:
print("train_df_x:")
print(train_X.shape)

print("train_df_y:")
print(train_y.shape)

print("test_df_x:")
print(test_X.shape)

print("test_df_y:")
print(test_y.shape)

train_df_x:
(3132, 8)
train_df_y:
(3132, 1)
test_df_x:
(1045, 8)
test_df_y:
(1045, 1)


In [356]:
def getAccuracy(testSet, predictions):

    correct = 0

    for x in range(len(testSet)):

        if testSet[x][-1] == predictions[x]:

            correct += 1

    return (correct/float(len(testSet))) * 100.0

In [357]:
from sklearn.linear_model import LogisticRegression
import math
from sklearn import metrics
from sklearn.metrics import accuracy_score, mean_absolute_error, median_absolute_error, mean_squared_error


#logistic regression
log_reg = LogisticRegression(multi_class='multinomial',solver ='newton-cg')
log_reg.fit(train_X, train_y)
print ("Intercept is ",log_reg.intercept_)
print("Coefficient is ",log_reg.coef_)
y_pred=log_reg.predict(test_X) 

# Accuracy Score
print("Accuracy of Logistic Regression is:", accuracy_score(test_y, y_pred))

#Mean Absolute Error 
mae=mean_absolute_error(test_y,y_pred);
print("MAE:"+str(mae))

#RMSE 
rmse = math.sqrt(mean_squared_error(test_y,y_pred))
print("RMSE:"+str(rmse))

#Median Absolute error
Medae=median_absolute_error(test_y,y_pred)
print("Median Absolute Error:"+str(Medae)) 

#Classification Report
print("Classification report for Test data %s:\n%s\n\n"
     % (log_reg, metrics.classification_report(test_y, y_pred)))

Intercept is  [ 2.49975834 -1.00850406 -1.49125428]
Coefficient is  [[ 0.32348155 -1.54749461 -2.08639723 -1.7553194  -2.35599186  4.33355641
  -0.6671625  -4.43501068]
 [-0.14213881  1.86873305  1.45182587  0.0130799  -0.96763201  1.91531126
   1.07611901 -0.47554634]
 [-0.18134273 -0.32123844  0.63457137  1.7422395   3.32362387 -6.24886767
  -0.40895652  4.91055702]]
Accuracy of Logistic Regression is: 0.6239234449760765
MAE:0.43732057416267944
RMSE:0.7482035902347646
Median Absolute Error:0.0
Classification report for Test data LogisticRegression(multi_class='multinomial', solver='newton-cg'):
              precision    recall  f1-score   support

           1       0.71      0.76      0.73       371
           2       0.49      0.44      0.46       322
           3       0.64      0.65      0.65       352

    accuracy                           0.62      1045
   macro avg       0.61      0.62      0.61      1045
weighted avg       0.62      0.62      0.62      1045





#### 5.	Create a neural network using pytorch to predict the same result as question 3. 

In [604]:
abalone_df = pd.read_csv(target_url,header=None, prefix="V")
abalone_df.columns = ['Sex', 'Length', 'Diameter', 'Height',
                   'Whole weight', 'Shucked weight',
                   'Viscera weight', 'Shell weight', 'Rings']
abalone_df = pd.DataFrame(abalone_df, columns=abalone_df.columns)
y = abalone_df.Rings

print(abalone_df.shape)
print(abalone_df.head())

(4177, 9)
  Sex  Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
4   I   0.330     0.255   0.080        0.2050          0.0895          0.0395   

   Shell weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
4         0.055      7  


In [605]:
# As Rings: +1.5 gives the age in years , So we will replace rings with age
abalone_df['Age'] = abalone_df['Rings'] + 1.5

#  Now we will drop Rings
# abalone_df.drop('Rings', axis=1, inplace=True)
abalone_df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Age
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,16.5
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,8.5
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,10.5
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,11.5
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,8.5


In [606]:
# One hot encoding the sex attribute.
df_dummies = pd.get_dummies(abalone_df['Sex'], prefix = "Sex_") # drop_first parameter is set to True, this means
                                                                           # that if there are n categories in the column,
                                                                           # n-1 columns are returned instead of n.

# Inserting dummy columns
for column in df_dummies.columns:
    abalone_df[column] = df_dummies[column]  # one-hot encoded columns are appended to the data frame
    
# Dropping the original column
abalone_df = abalone_df.drop(columns = ['Sex'])   

In [607]:
print(abalone_df)

      Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0      0.455     0.365   0.095        0.5140          0.2245          0.1010   
1      0.350     0.265   0.090        0.2255          0.0995          0.0485   
2      0.530     0.420   0.135        0.6770          0.2565          0.1415   
3      0.440     0.365   0.125        0.5160          0.2155          0.1140   
4      0.330     0.255   0.080        0.2050          0.0895          0.0395   
...      ...       ...     ...           ...             ...             ...   
4172   0.565     0.450   0.165        0.8870          0.3700          0.2390   
4173   0.590     0.440   0.135        0.9660          0.4390          0.2145   
4174   0.600     0.475   0.205        1.1760          0.5255          0.2875   
4175   0.625     0.485   0.150        1.0945          0.5310          0.2610   
4176   0.710     0.555   0.195        1.9485          0.9455          0.3765   

      Shell weight  Rings   Age  Sex__F

In [608]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = abalone_df.drop(['Rings','Age'], axis = 1)
y = abalone_df.drop(['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight',
              'Shell weight', 'Age','Sex__F','Sex__I','Sex__M'], axis = 1)

# y[columnn] = pd.DataFrame(y)

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

# #Standardize the data
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [609]:
# y = abalone_df.drop(['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight',
#               'Shell weight', 'Rings','Sex__F','Sex__I','Sex__M'], axis = 1)

In [610]:
print(y)

      Rings
0        15
1         7
2         9
3        10
4         7
...     ...
4172     11
4173     10
4174      9
4175     10
4176     12

[4177 rows x 1 columns]


In [611]:
print("train_df_x:")
print(X_train.shape)

print("train_df_y:")
print(y_train.shape)

print("test_df_x:")
print(X_test.shape)

print("test_df_y:")
print(y_test.shape)

train_df_x:
(2923, 10)
train_df_y:
(2923, 1)
test_df_x:
(1254, 10)
test_df_y:
(1254, 1)


In [613]:
# Import Dependencies
import torch.nn as nn
import torch
import torch.nn.functional as F # Activation function

# Create tensors 
X_train = torch.Tensor(X_train) 
X_test = torch.Tensor(X_test)

y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

# test_tensor = torch.Tensor(test.values)
print(X_train)

TypeError: new(): data must be a sequence (got builtin_function_or_method)

In [577]:
#artificial neural network
class ANN_Model(nn.Module):
    def __init__(self, input_features=10,hidden1=100,hidden2=100,out_features=2):
        super().__init__()
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
#         self.layer_3_connection = nn.Linear(hidden2, hidden3)
        self.out = nn.Linear(hidden2, out_features)
        
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.softmax(self.layer_2_connection(x))
#         x = F.relu(self.layer_3_connection(x))
        x = self.out(x)
        return x

In [578]:
torch.manual_seed(69)

#create instance of model
ann = ANN_Model()

In [579]:
#loss function
loss_function = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.Adam(ann.parameters(), lr=0.01)

In [580]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = ann.forward(X_train)
    loss = loss_function(y_train, y_pred)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss}')
        
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() 
    optimizer.step() #perform one optimization step each epoch

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [294]:
#predictions
y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = ann(data)
        y_pred.append(prediction.argmax()) 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x10 and 9x100)

In [ ]:
print(y_pred)

In [168]:
# Import mean_squared_error as MSE
from sklearn.metrics import mean_squared_error as MSE

# Evaluate the test set RMSE
rmse_test = MSE(y_test, y_pred)**(1/2)

ValueError: y_true and y_pred have different number of output (3!=1)

In [169]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets

In [ ]:
######################################################################################################################

In [222]:
df = pd.read_csv(target_url,header=None, prefix="V")
df.columns = ['Sex', 'Length', 'Diameter', 'Height',
                   'Whole weight', 'Shucked weight',
                   'Viscera weight', 'Shell weight', 'Rings']

print(df.head())

  Sex  Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
4   I   0.330     0.255   0.080        0.2050          0.0895          0.0395   

   Shell weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
4         0.055      7  


In [223]:
import warnings
warnings.filterwarnings('ignore')

df["M"] = np.nan
df["F"] = np.nan
df["I"] = np.nan
columnName='Sex'
for i in range (len(df[columnName])):
    if df[columnName][i]=='M':
        df['M'][i]=1
        df['F'][i]=0
        df['I'][i]=0
    elif df[columnName][i]=='F':
        df['M'][i]=0
        df['F'][i]=1
        df['I'][i]=0
    elif df[columnName][i]=='I' :
        df['M'][i]=0
        df['F'][i]=0
        df['I'][i]=1
df=df.drop(['Sex'],axis=1)

In [224]:
print(df)

      Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0      0.455     0.365   0.095        0.5140          0.2245          0.1010   
1      0.350     0.265   0.090        0.2255          0.0995          0.0485   
2      0.530     0.420   0.135        0.6770          0.2565          0.1415   
3      0.440     0.365   0.125        0.5160          0.2155          0.1140   
4      0.330     0.255   0.080        0.2050          0.0895          0.0395   
...      ...       ...     ...           ...             ...             ...   
4172   0.565     0.450   0.165        0.8870          0.3700          0.2390   
4173   0.590     0.440   0.135        0.9660          0.4390          0.2145   
4174   0.600     0.475   0.205        1.1760          0.5255          0.2875   
4175   0.625     0.485   0.150        1.0945          0.5310          0.2610   
4176   0.710     0.555   0.195        1.9485          0.9455          0.3765   

      Shell weight  Rings    M    F    

In [210]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(10,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='relu'))
model.summary()

# early_stopping_monitor = EarlyStopping(patience=2)

# model.compile(optimizer = 'Adam', loss = 'CategoricalCrossentropy', metrics = ['accuracy'])

# history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 256, callbacks = [early_stopping_monitor])


model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
history = model.fit(x_train,y_train,batch_size=100,epochs=5,verbose=1)
test=model.evaluate(x_test, y_test, verbose=1)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 256)               2816      
                                                                 
 dense_14 (Dense)            (None, 256)               65792     
                                                                 
 dense_15 (Dense)            (None, 256)               65792     
                                                                 
 dense_16 (Dense)            (None, 256)               65792     
                                                                 
 dense_17 (Dense)            (None, 1)                 257       
                                                                 
Total params: 200,449
Trainable params: 200,449
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
30/30 [==============================]

#### 6.	Compare the performance of the neural networks to the other model you created. Which performed better? Why do you think that is?
